<a href="https://colab.research.google.com/github/DousMoon/ml0804/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0804/raw/master/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0804/raw/master/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

In [ ]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df

In [ ]:
# concat: axis!
import pandas as pd
datas = pd.concat([train_df,test_df]).reset_index(drop=True)
datas = datas.drop(["PassengerId","Survived"],axis=1)
datas

In [ ]:
s = datas.isna().sum() # isna沒有值=True
s[s>0].sort_values() # 大到小:s[s>0].sort_values(ascending = False)

In [ ]:
# unique/value_counts
# 類別請補最常出現
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
# 數值補中位數
med = datas.median().drop(["Pclass"])
datas.fillna(med)

In [ ]:
# apply:
def cabin(s):
  if not pd.isna(s):
    return s[0]
  else:
    return None
datas["Cabin"] = datas["Cabin"].apply(cabin)

In [ ]:
v = datas["Ticket"].value_counts()
def ticket(s):
    global v
    return v[s]
datas["Ticket"] = datas["Ticket"].apply(ticket)